In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-dbscandata/new_datadbscan.csv


In [2]:
! pip install umap
! pip install umap-learn


  Preparing metadata (setup.py) ... - \ done
  Created wheel for umap: filename=umap-0.1.1-py3-none-any.whl size=3543 sha256=f30bcffd52f74a35c05f84ace248f5c077a1ba375292693d85d1c31b8a988b47
  Stored in directory: /root/.cache/pip/wheels/15/f1/28/53dcf7a309118ed35d810a5f9cb995217800f3f269ab5771cb
Successfully built umap
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 3.8 MB/s eta 0:00:00


In [3]:
import gensim
from gensim import corpora
from gensim.models import CoherenceModel
from gensim.models import LdaModel
from tqdm import tqdm
from nltk.tokenize import word_tokenize
import collections
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import product
import umap.umap_ as umap
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score

In [4]:
data = pd.read_csv('/kaggle/input/new-dbscandata/new_datadbscan.csv')
data

,Unnamed: 0,Istilah,Asal Daerah,Deskripsi,Judul Esai,Link Esai,Diambil dari Penulis Author(s),Related to SDGs,Link Foto (opsional),Essay Kontes Ke,Isi,Deskripsi_Cleaned,Stopword_Count
0,0,Ampa Fare,NTB,Tradisi musim panen dimana padi yang telah dik...,Parafu Ro Waro dalam Pusaran Mata Air: Menghid...,https://drive.google.com/open?id=1cKyTeGXRZxck...,Kasmiatun,No. 6,NaN,No. 3,"Populasi penduduk dunia yang kian bertambah, d...",tradisi musim panen mana padi kumpul angkut ru...,"Counter({'yang': 1, 'telah': 1, 'ke': 1, 'atau..."
1,1,Bau Nyale,NTB,Tradisi menangkap cacing laut yang dilakukan m...,Parafu Ro Waro dalam Pusaran Mata Air: Menghid...,https://drive.google.com/open?id=1cKyTeGXRZxck...,Kasmiatun,No. 6,NaN,No. 3,"Populasi penduduk dunia yang kian bertambah, d...",tradisi tangkap cacing laut masyarakat suku sa...,"Counter({'yang': 2, 'dilakukan': 1, 'setiap': ..."
2,2,Beas Perelek,Jawa Barat,Tradisi patungan/urunan warga dalam bentuk sum...,Mewujudkan Dunia Tanpa Kelaparan (No Hunger) M...,https://drive.google.com/open?id=1l8i3YsUEDwKx...,Heri Heryana,"No. 2, 3, & 15",NaN,No. 3,Tahun 2022 merupakan tahun ke-7 pelaksanaan pr...,tradisi patunganurunan warga bentuk sumbang be...,"Counter({'dalam': 4, 'yang': 4, 'atau': 3, 'da..."
3,3,Dawan,"Pulau Timor, NTT","sapaan orang bukan ""Atoin Meto"" kepada masyara...",Tradisi Usá: Kearifan Lokal Atoin Meto (Dawan)...,https://drive.google.com/open?id=1YKwwxv3VXROf...,Jondry Siki,NaN,NaN,No. 3,Dawan adalah salah satu etnis yang mendiami Pu...,sapa orang atoin meto masyarakat tutur uab met...,"Counter({'yang': 2, 'kata': 2, 'menjadi': 2, '..."
4,4,Dusung,Maluku,Merupakan kearifan lokal masyarakat Maluku yan...,Belajar dari Dusung Konsep Agroforestri Intens...,https://drive.google.com/open?id=1p11P4IR7H15w...,Moch Sofiyulloh,"No. 2, 3, & 15",NaN,No. 3,"Dunia saat ini berkembang sangat pesat, dengan...",arif lokal masyarakat malu jalan turun turun p...,"Counter({'yang': 2, 'dengan': 2, 'merupakan': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,392,Ngahuma,NaN,Ngahuma adalah sistem pertanian tradisional ma...,Pertanian Berbasis Kearifan Budaya hingga Bang...,https://drive.google.com/open?id=1YW0dtNRJdnhI...,Muhammad Naufal Rizqulloh,"No. 1,2,3,4",NaN,Ke 3,Kegiatan bertani pada hakikatnya merupakan keg...,ngahuma sistem tani tradisional masyarakat sun...,"Counter({'yang': 3, 'dari': 3, 'dan': 3, 'di':..."
382,393,Jaranan Buto,Jawa Timur,Jaranan Buto adalah pertunjukan seni khas Bany...,Peran dan Keterlibatan Perempuan dalam Kesenia...,https://drive.google.com/open?id=19740DnCJa3RQ...,Julita Hasanah,No. 5,NaN,Ke 3,Terlahir sebagai perempuan asli Jawa Timur mem...,jaranan buto tunjuk seni khas banyuwangi kaya ...,"Counter({'dalam': 4, 'dan': 3, 'ini': 3, 'yang..."
383,394,Mokarawo,Gorontalo,Mokarawo merupakan seni sulam tradisional khas...,Fusi Tradisi Huyula dan Mokarawo sebagai Prins...,https://drive.google.com/open?id=1a1Ks8ce9vy22...,Muhammad Rizq Gobel,"No. 1,4,5,8",NaN,Ke 3,Peradaban manusia yang semakin maju dan tidak ...,mokarawo seni sulam tradisional khas gorontalo...,"Counter({'dan': 4, 'yang': 3, 'para': 2, 'dala..."
384,395,Wija pattola palallo,Sulawesi Selatan,Wija pattola palallo adalah prinsip kearifan l...,Wija Pattola Palallo: Motivasi Orang Tua Bugis...,https://drive.google.com/open?id=1K5io345ZMTfW...,Adnan Achiruddin Saleh,NaN,NaN,Ke 3,Sebuah percakapan tentang masa depan; Melo ka ...,wija pattola palallo prinsip arif lokal masyar...,"Counter({'yang': 4, 'untuk': 3, 'dan': 3, 'ini..."


In [5]:
data['Tokenized'] = data['Deskripsi_Cleaned'].apply(lambda x: word_tokenize(x))
print(data[['Deskripsi_Cleaned', 'Tokenized']])

                                     Deskripsi_Cleaned  \
0    tradisi musim panen mana padi kumpul angkut ru...   
1    tradisi tangkap cacing laut masyarakat suku sa...   
2    tradisi patunganurunan warga bentuk sumbang be...   
3    sapa orang atoin meto masyarakat tutur uab met...   
4    arif lokal masyarakat malu jalan turun turun p...   
..                                                 ...   
381  ngahuma sistem tani tradisional masyarakat sun...   
382  jaranan buto tunjuk seni khas banyuwangi kaya ...   
383  mokarawo seni sulam tradisional khas gorontalo...   
384  wija pattola palallo prinsip arif lokal masyar...   
385  ngumbung tradisi masyarakat karo laksana enam ...   

                                             Tokenized  
0    [tradisi, musim, panen, mana, padi, kumpul, an...  
1    [tradisi, tangkap, cacing, laut, masyarakat, s...  
2    [tradisi, patunganurunan, warga, bentuk, sumba...  
3    [sapa, orang, atoin, meto, masyarakat, tutur, ...  
4    [arif, lokal,

In [6]:
id2word = corpora.Dictionary(data['Tokenized'])
corpus = [id2word.doc2bow(text) for text in data['Tokenized']]

In [7]:
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=22,
    random_state=100,
    update_every=1,
    chunksize=10,
    passes=10,
    alpha=0.01,
    eta=0.91,
    per_word_topics=True
)

In [8]:
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(9, '0.000*"terubuk" + 0.000*"bengkal" + 0.000*"riau" + 0.000*"tenualosa" + 0.000*"selat"')
(20, '0.000*"anyam" + 0.000*"mamatik" + 0.000*"madihin" + 0.000*"pamadihinan" + 0.000*"langsat"')
(11, '0.002*"gempa" + 0.001*"beton" + 0.001*"konstruksi" + 0.001*"rancang" + 0.001*"cedera"')
(8, '0.022*"masyarakat" + 0.019*"tradisi" + 0.011*"budaya" + 0.009*"lokal" + 0.008*"tradisional"')
(18, '0.004*"pompong" + 0.003*"buang" + 0.003*"anambas" + 0.002*"motor" + 0.002*"ancak"')
(13, '0.008*"tari" + 0.003*"besut" + 0.002*"rusmini" + 0.002*"juang" + 0.002*"boyolali"')
(4, '0.003*"kitab" + 0.002*"karya" + 0.002*"sastra" + 0.002*"pakubuwana" + 0.002*"wulangreh"')
(16, '0.001*"tawon" + 0.001*"sarang" + 0.001*"guyub" + 0.001*"mangunweni" + 0.001*"tempe"')
(2, '0.008*"karo" + 0.007*"tengger" + 0.003*"bromo" + 0.003*"uis" + 0.003*"kasada"')
(17, '0.000*"ado" + 0.000*"ndak" + 0.000*"sanskerta" + 0.000*"menjamim" + 0.000*"makanann"')
(7, '0.002*"anyam" + 0.001*"mamatik" + 0.001*"ritus" + 0.001*"estetis" +

In [9]:
doc_index = 0
doc_bow = corpus[doc_index]
topic_distribution = lda_model.get_document_topics(doc_bow)

print(f"Distribusi topik untuk dokumen {doc_index + 1}:")
for topic_id, prob in topic_distribution:
    print(f"Topik {topic_id}: Probabilitas {prob:.4f}")

Distribusi topik untuk dokumen 1:
Topik 3: Probabilitas 0.1627
Topik 8: Probabilitas 0.8296


In [10]:
topic_distributions = []
for doc_bow in corpus:
    doc_topics = lda_model.get_document_topics(doc_bow, minimum_probability=0)
    topic_dist = [prob for _, prob in doc_topics]
    topic_distributions.append(topic_dist)

topic_matrix = pd.DataFrame(topic_distributions)
print(topic_matrix)

           0         1         2         3         4         5         6   \
0    0.000382  0.000382  0.000382  0.159904  0.000382  0.000382  0.000382   
1    0.000819  0.000819  0.000819  0.000819  0.000819  0.000819  0.000819   
2    0.000199  0.000199  0.000199  0.000199  0.000199  0.000199  0.000199   
3    0.000581  0.000581  0.000581  0.000581  0.000581  0.000581  0.000581   
4    0.000354  0.000354  0.000354  0.992556  0.000354  0.000354  0.000354   
..        ...       ...       ...       ...       ...       ...       ...   
381  0.000172  0.000172  0.000172  0.951332  0.000172  0.000172  0.000172   
382  0.000161  0.000161  0.000161  0.000161  0.000161  0.000161  0.000161   
383  0.000181  0.000181  0.000181  0.000181  0.000181  0.000181  0.000181   
384  0.000161  0.000161  0.000161  0.000161  0.000161  0.000161  0.000161   
385  0.000172  0.000172  0.000172  0.922922  0.000172  0.000172  0.000172   

           7         8         9   ...        12        13        14  \
0  

In [11]:
n_neighbors_range = [5, 10, 15, 17, 20]
min_dist_range = [0.01, 0.1, 0.104, 0.2, 0.5]
eps_range = [0.5, 0.7, 0.9, 0.926, 1.0]
min_samples_range = [5, 10, 20, 42, 50]

results = []
for n_neighbors, min_dist, eps, min_samples in tqdm(product(n_neighbors_range, min_dist_range, eps_range, min_samples_range)):
    
    umap_model = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_dist, n_components=2, random_state=42)
    embedding = umap_model.fit_transform(topic_matrix)
    dbscan_model = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan_model.fit_predict(embedding)
    if len(set(labels)) > 1:
        silhouette_avg = silhouette_score(embedding, labels)
        results.append({
            'n_neighbors': n_neighbors,
            'min_dist': min_dist,
            'eps': eps,
            'min_samples': min_samples,
            'silhouette_score': silhouette_avg
        })

results_df = pd.DataFrame(results)
top_results = results_df.nlargest(5, 'silhouette_score')
print("Top 5 kombinasi terbaik berdasarkan silhouette score:")
print(top_results)

0it [00:00, ?it/s]/opt/conda/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
1it [00:08,  8.56s/it]/opt/conda/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
2it [00:09,  3.88s/it]/opt/conda/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
3it [00:09,  2.38s/it]/opt/conda/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
4it [00:10,  1.64s/it]/opt/conda/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
5it [00:10,  1.24s/it]/opt/conda/lib/python3.10/site-packages/um

Top 5 kombinasi terbaik berdasarkan silhouette score:
     n_neighbors  min_dist    eps  min_samples  silhouette_score
5              5      0.01  0.900            5          0.700038
8              5      0.01  0.926            5          0.700038
11             5      0.01  1.000            5          0.700038
220           17      0.10  0.900            5          0.698550
221           17      0.10  0.900           10          0.698550
